# Time Bar

In [1]:
import gc
import glob
import os
from typing import Dict, List

import dask.bag as db
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd

In [2]:
from tqdm.notebook import tqdm
tqdm.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# from pandarallel import pandarallel
# pandarallel.initialize()

In [4]:
from utils import aggregate, aggregate_trade, convert_to_bar

In [5]:
aggregate(pd.Series([3,2,1,4,5,6]))

{'open': 3, 'high': 6, 'low': 1, 'close': 6, 'mean': 3.5, 'median': 3.5}

In [8]:
def generate_time_bars(hdf_file: str, bar_size: np.int64, output_file: str)->None:
    """Build time bars.
    Args:
        df: per trade data.
        bar_size: time bar size, in milliseconds.
        output_dir: Output directory.
    Returns:
        Time bars.
  """
    df = pd.read_hdf(hdf_file)
    df['timestamp']=df['timestamp'].astype(np.int64) // int(1e6)
    df['timestamp_begin']=df['timestamp'] // bar_size * bar_size
    grouped = df.groupby('timestamp_begin')

    series = grouped.apply(lambda df: convert_to_bar('TimeBar', bar_size, df))  # parallel_apply, from pandarallel
    
    del df
    gc.collect()
    
    bars = pd.DataFrame(list(series))

    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    bars.to_hdf(
        output_file,
        key=os.path.basename(output_file)[0:-len('.hdf5')].replace('.', '__'),
        mode='w',
        complevel=9)

In [9]:
# generate_time_bars('/data/hdf5/OKEx.Swap.BTC_USDT.hdf5', 10000,
#                    '/data/bars/TimeBar/10000/TimeBar.10000.OKEx.Swap.BTC_USDT.hdf5')

In [10]:
def generate_multi(hdf5_files: List[str], bar_sizes: List[int], output_dir: str)->None:
    file_bar_sizes = [(file, bar_size) for file in hdf5_files for bar_size in bar_sizes]
    with ProgressBar():
        db.from_sequence(file_bar_sizes).map(
            lambda t: generate_time_bars(t[0], t[1], os.path.join(
                output_dir, str(t[1]), f'TimeBar.{t[1]}.{os.path.basename(t[0])}'))).compute()

In [11]:
hdf_files = glob.glob('/data/hdf5/*BTC_USD*.hdf5')

In [12]:
hdf_files

['/data/hdf5/OKEx.Swap.BTC_USD.hdf5',
 '/data/hdf5/Bitfinex.Spot.BTC_USDT.hdf5',
 '/data/hdf5/MXC.Spot.BTC_USDT.hdf5',
 '/data/hdf5/Huobi.Swap.BTC_USD.hdf5',
 '/data/hdf5/Newdex.Spot.BTC_USDT.hdf5',
 '/data/hdf5/Bitfinex.Swap.BTC_USDT.hdf5',
 '/data/hdf5/WhaleEx.Spot.BTC_USDT.hdf5',
 '/data/hdf5/Binance.Spot.BTC_USDT.hdf5',
 '/data/hdf5/Binance.Swap.BTC_USDT.hdf5',
 '/data/hdf5/OKEx.Spot.BTC_USDT.hdf5',
 '/data/hdf5/Bitfinex.Spot.BTC_USD.hdf5',
 '/data/hdf5/Huobi.Spot.BTC_USDT.hdf5',
 '/data/hdf5/CoinbasePro.Spot.BTC_USD.hdf5',
 '/data/hdf5/Bitstamp.Spot.BTC_USD.hdf5',
 '/data/hdf5/BitMEX.Swap.BTC_USD.hdf5',
 '/data/hdf5/OKEx.Swap.BTC_USDT.hdf5',
 '/data/hdf5/Kraken.Spot.BTC_USDT.hdf5',
 '/data/hdf5/Kraken.Spot.BTC_USD.hdf5']

In [13]:
generate_multi(
    hdf_files,
    list(map(lambda x: x* 1000, [10, 60, 180, 300, 900, 1800, 3600, 14400])),
    '/data/bars/TimeBar',
)

[########################################] | 100% Completed | 28min  1.1s


## References

* [Tick, Volume, Dollar Volume Bars.ipynb](https://github.com/BlackArbsCEO/Adv_Fin_ML_Exercises/blob/master/notebooks/Tick%2C%20Volume%2C%20Dollar%20Volume%20Bars.ipynb)